## ทำการ web scraping

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver
# driver set uo
driver = driversetup()
# ดึง comment มาเก็บเป็น list ใน review
reviews = []
for i in range(0,80,10):
    driver.get("https://th.tripadvisor.com/Attraction_Review-g293919-d9650315-Reviews-or"+str(i)+"-Jomtien_Beach_Night_Market-Pattaya_Chonburi_Province.html")
    print(i, end=" ")
    time.sleep(5)
    lis_reviews = driver.find_elements(By.CLASS_NAME, "yCeTE")[0:]
    print(len(lis_reviews))
    for review in lis_reviews:
        reviews.append(review.text)

0 20
10 20
20 20
30 20
40 20
50 20
60 20
70 4


In [9]:
len(reviews)

144

## สร้าง datframe

In [10]:
import pandas as pd
df = pd.DataFrame(reviews, columns=["review"])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  144 non-null    object
dtypes: object(1)
memory usage: 1.2+ KB


In [11]:
df.head()

,review
0,Pattaya for eat
1,ที่นี่ติดชายหาดเลย มีบาร์ให้นั่งกินอาหารมองเห็...
2,E
3,ที่นี่ถ้า5ทุ่มเป๊ะปิดเลย แถมยามยังมาไล่แบบกินเ...
4,ตลาดกลางคืน ริมชายหาด สินค้าหลากหลาย


## preprocess_text (Normalize, Word Tokenize, Remove stopwords,  Remove any non-Thai characters)

In [15]:
from pythainlp import corpus
from pythainlp.util import normalize
from collections import Counter
import deepcut

In [16]:
def preprocess_text(text):
    # Normalize the text
    normalized_text = normalize(text)
    # Tokenize the normalized text into words using deepcut
    words = deepcut.tokenize(normalized_text)
    # Remove any stopwords
    words = [word for word in words if word not in corpus.thai_stopwords()]
    # Remove any non-Thai characters
    thai_words = [word for word in words if all(c >= 'ก' and c <= '๛' for c in word)]
    return thai_words

In [17]:
df['processed_text'] = df['review'].apply(preprocess_text)

10/10 [==============================] - 0s 9ms/step


In [18]:
df.head()

,review,processed_text
0,Pattaya for eat,[]
1,ที่นี่ติดชายหาดเลย มีบาร์ให้นั่งกินอาหารมองเห็...,"[ติด, ชาย, หาด, บาร์, นั่ง, กิน, อาหาร, วิวทะเ..."
2,E,[]
3,ที่นี่ถ้า5ทุ่มเป๊ะปิดเลย แถมยามยังมาไล่แบบกินเ...,"[ทุ่มเป๊ะ, แถม, ยาม, ไล่, กิน, นั่ง, กิน, หล่ะ..."
4,ตลาดกลางคืน ริมชายหาด สินค้าหลากหลาย,"[ตลาด, กลาง, คืน, ริม, ชายหาด, สินค้า, หลากหลาย]"


## export to csv

In [19]:
#df.to_csv("Jomtien_Beach_Night_Market_Processed.csv")